<a href="https://colab.research.google.com/github/ElenaPribylova/-10/blob/main/3_%D0%BA%D0%B5%D0%B9%D1%81_%D0%9F%D0%95%D0%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-10-11 10:31:17--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-10-11 10:31:17--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-10-11 10:31:17 (90.3 M

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# Подготовим таблицы к работе

with open('registrations.csv') as f:
    reg_rows = f.readlines()[1:]  # пропускаем строку заголовка

# собираем первый файл в список словарей
registrations = []
for row in reg_rows:
    user_id = int(row.split(";")[0])
    registration_date = row.split(";")[1].strip()
    registrations.append({"user_id": user_id, "registration_date": registration_date})

# то же самое с вторым файлом
with open('entries.csv') as f:
    entries_rows = f.readlines()[1:]

entries = []
for row in entries_rows:
    user_id = int(row.split(";")[0])
    entry_date = row.split(";")[1].strip()
    entries.append({"user_id": user_id, "entry_date": entry_date})

In [ ]:
# Уберем записи о входах, где один юзер зашел дважды за день

users_on_day = set()

filtered_entries = []

for entry in entries:
    user_id = entry['user_id']
    entry_date = entry['entry_date']

    if (user_id, entry_date) in users_on_day:
        continue
    else:
        users_on_day.add((user_id, entry_date))
        filtered_entries.append(entry)

# Отфильтрованные входы теперь в `filtered_entries`

In [2]:
import pandas as pd

january_users = df[
    (df['registration_date'].dt.month == 1)
]
# Получаем общее количество январских пользователей
total_january_users = january_users['user_id'].nunique()

# Вычисляем дату 15-го дня для каждого пользователя
january_users['day_15'] = january_users['registration_date'] + pd.Timedelta(days=15)

# Находим активных пользователей на 15-й день
entry_date_users_day_15 = january_users[
    january_users['entry_date'].isin(january_users['day_15'])
]['user_id'].nunique()

# Рассчитываем Retention 15 дня
retention_15_day = (active_users_day_15 / total_january_users) * 100

# Округляем результат до 5 знаков после запятой
retention_15_day = round(retention_15_day, 5)

print(f"Retention 15 дня для январских пользователей: {retention_15_day}%")



In [4]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'retention_15_day' is not defined

### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# создадим множество для хранения юзеров зашедших в 30 день или после
rolling_retention_rates = set()

for i in days_since_registration:
    if i['days_since_registration'] >= 30:
        rolling_retention_rates.add(i['user_id'])

# считаем rolling retention
rolling_retention = round(len(rolling_retention_rates)/len(january_registrations) * 100, 5)
round(rolling_retention, 5)

29.06977

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [ ]:
# объединим регистрации и отфильтрованные входы по user_id
mergedallusers = []
for registration in registrations:
    for entry in filtered_entries:
        if entry["user_id"] == registration["user_id"]:
            mergedallusers.append({"user_id": entry["user_id"], "registration_date": registration["registration_date"], "entry_date": entry["entry_date"]})

# считаем количество дней с регистрации для каждого входа каждого пользователя
days_since_registration_all = []
for m in mergedallusers:
    from datetime import datetime
    registration_date = datetime.strptime(m["registration_date"], "%Y-%m-%d")
    entry_date = datetime.strptime(m["entry_date"], "%Y-%m-%d")
    days = (entry_date - registration_date).days
    days_since_registration_all.append({"user_id": m["user_id"], "days_since_registration": days})

counts = {}
for entry in days_since_registration_all:
    days_since_registration = entry['days_since_registration']
    if days_since_registration not in counts:
        counts[days_since_registration] = 0
    counts[days_since_registration] += 1
each_day_ret = dict(sorted(counts.items()))

day_zero_count = each_day_ret[0]
lifetime = 0

for day, count in each_day_ret.items():
    lifetime += (count / day_zero_count)

lifetime = round(lifetime, 5)

In [5]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'lifetime' is not defined

### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [8]:
cr_29 = set()
# churn rate n дня - юзеры которые на n день отвалились от продукта.
# Соответственно, проверяем были ли у них заходы после n дня,
# если были - добавляем в сет - это не отвалившиеся юзеры.
for i in days_since_registration_all:
    if i['days_since_registration'] >= 29:
        cr_29.add(i['user_id'])

# теперь вычитаем из всех зарегистрированных юзеровв, оставшихся юзеров и делим на зарегистрированных

churn_29 = (len(registrations)-len(cr_29))/len(registrations)
churn_29

NameError: name 'days_since_registration_all' is not defined

In [7]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'churn_29' is not defined

### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [ ]:
# получаем входы за декабрь
december_user_entries = []
for i in filtered_entries:
    if i['entry_date'][:4] == "2021" and i['entry_date'][5:7] == '12':
        december_user_entries.append(i)

In [ ]:
# из входов за декбрь рассчитываем mau

december_2022_mau = set()
for i in december_user_entries:
    december_2022_mau.add(i['user_id'])

dec_mau = len(december_2022_mau)

In [ ]:
# тоже самое для wau

december_wau = set()
for i in december_user_entries:
    if i['entry_date'][-2:] in ('25', '26', '27', '28', '29', '30', '31'):
        december_wau.add(i['user_id'])

dec_wau = len(december_wau)

In [ ]:
# и dau

december_dau = set()
for i in december_user_entries:
    if i['entry_date'][-2:] == '31':
        december_dau.add(i['user_id'])

dec_dau = len(december_dau)

In [ ]:
from collections import defaultdict

# Создаем defaultdict для хранения уникальных юзеров каждог месяца
counts_avg = defaultdict(set)

# Пройдем по входам
for line in entries_rows:
    # Выделим user_id и дату входа
    user_id, entry_date = line.strip().split(';')

    # Вытащим год и месяц
    year, month = entry_date.split('-')[:2]

    # Используем год и месяц в качестве ключа
    key = f"{year}-{month}"
    counts_avg[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts_avg.items():
    count_of_unique_user_ids[key] = len(value)

In [ ]:
# считаем усредненный mau

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_mau = round(sum/len(count_of_unique_user_ids), 5)

In [ ]:
from collections import defaultdict

from datetime import datetime

counts = defaultdict(set)

for line in entries_rows:
    user_id, entry_date = line.strip().split(';')

    # Преобразуем строку с датой в объект datetime
    date_obj = datetime.strptime(entry_date, '%Y-%m-%d')

    # Получаем номер недели в году и год с помощью метода isocalendar()
    year, week_num, _ = date_obj.isocalendar()

    # Формируем ключ в формате "ГОД-НОМЕР_НЕДЕЛИ"
    key = f"{year}-{week_num}"
    counts[key].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)


In [ ]:
# считаем усредненный wau

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_wau = round(sum/len(count_of_unique_user_ids), 5)

In [ ]:
# Все то же самое для каждого дня
counts = defaultdict(set)

for line in entries_rows:
    user_id, entry_date = line.strip().split(';')

    # В качестве ключа используем дату входа
    counts[entry_date].add(user_id)

# Конвертируем множество пользователей в количество уникальных пользователей
count_of_unique_user_ids = {}
for key, value in counts.items():
    count_of_unique_user_ids[key] = len(value)

sum = 0
for key, value in count_of_unique_user_ids.items():
    sum += value

avg_dau = round(sum/len(count_of_unique_user_ids), 5)

In [9]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

NameError: name 'dec_mau' is not defined

In [ ]:
# Ваше решение


dec_wau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

In [ ]:
# Ваше решение


dec_dau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [ ]:
# Ваше решение


avg_mau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

In [ ]:
# Ваше решение


avg_wau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

In [ ]:
# Ваше решение


avg_dau

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')